In [119]:
import jieba
import time
import csv
import re
import jieba
import jieba.analyse
from snownlp import SnowNLP
import requests
import plotly.graph_objs as go
import plotly
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = 'all'
import matplotlib.pyplot as plt
from matplotlib import pylab
import warnings
import os
import json
ak = 'YiqcrKeivYMmuuiKVj0xUI4eoVosfav6'
'''
half_data得到了删除了坐标空值，
清洗后的数据得到了坐标
主文档文件缺判断缺评分.csv为删除海外，商业机构，新闻媒体后的文件，但未评分，未判断
'''

In [13]:
def get_position(position, i):
    baiduUrl = 'http://api.map.baidu.com/reverse_geocoding/v3/?ak=' + ak + '&output=json&coordtype=wgs84ll&location=' + position[
        1] + ',' + position[0]
    try:
        req = requests.get(baiduUrl)
        content = req.text
        content = content.replace("renderReverse&&renderReverse(", "")
        baiduAddr = json.loads(content)
    except:
        print(str(i) + '出现错误')
    try:
        province = baiduAddr["result"]["addressComponent"]["province"]
        city = baiduAddr["result"]["addressComponent"]["city"]
    except:
        province = None
        city = None

    return city, province

# half_data依据

In [19]:
def city_condition(name):
    # 创建csv
    _ = open("D:\\IDEA\\情感分析论文\\half_data\\" + name + '_city' + '.csv',
             'w',
             encoding='utf-8')
    _.close()
    # 创建写入器
    csvfile = open("D:\\IDEA\\情感分析论文\\half_data\\" + name + '_city' + '.csv',
                   'a+',
                   newline='',
                   encoding='utf-8')
    # a+表示以追加模式写入，如果用w会覆盖掉原来的数据
    csv_write = csv.writer(csvfile)

    # 读取并写入
    f = open('D:\\IDEA\\情感分析论文\\' + name + '\\' + name + '.csv',
             encoding='utf-8')
    csv_read = csv.reader(f)
    for line in csv_read:
        if len(line[-1]) != 0:
            _ = csv_write.writerow(line)
    csvfile.close()
    f.close()

# CSV版本

In [8]:
def CSV_condition(line, NUM=50):
    try:
        condition_num_lower = np.array(line[3:6], dtype=int).sum() > 2
        condition_num_upper = ((int(line[3]) < NUM) & (int(line[4]) < NUM) &
                               (int(line[5]) < NUM))

        condition1 = ((len(line[6]) < 100) & (len(line[7]) == 0))
        condition2 = ((len(line[6]) < 200) & (len(line[7]) != 0))
        condition_post = (condition2 | condition1)

        #         condition_city=len(line[-1])!=0

        condition = (condition_num_lower & condition_num_upper & condition_post
                     )  #&condition_city)
        return condition
    except:
        return False

def data_clean(name):
    _ = open("D:\\IDEA\\情感分析论文\\清洗后的数据\\" + name + '_cleaned' + '.csv',
             'w',
             encoding='utf-8')
    _.close()
    print(name + '_clean.csv已创建')

    # 创建写入器
    csvfile = open("D:\\IDEA\\情感分析论文\\清洗后的数据\\" + name + '_cleaned' + '.csv',
                   'a+',
                   newline='',
                   encoding='utf-8')
    # a+表示以追加模式写入，如果用w会覆盖掉原来的数据
    csv_write = csv.writer(csvfile)

    # 读取并写入
    print(name + '_clean.csv正在写入')
    f = open('D:\\IDEA\\情感分析论文\\' + 'half_data' + '\\' + name + '_city.csv',
             encoding='utf-8')
    csv_read = csv.reader(f)
    for i, line in enumerate(csv_read):
        if CSV_condition(line):
            city, province = get_position(line[-1].split(','), i)
            NEW_LINE = line + [city] + [province]
            _ = csv_write.writerow(NEW_LINE)
        if i % 10000 == 0:
            print('第%d条' % i)
    csvfile.close()
    f.close()
    print(name + '_clean.csv写入完成')

# file_name = ['2020-01']
# for name in file_name:
#     file_start_time = time.time()
#     data_clean(name)
#     file_end_time = time.time()
#     print('清洗' + name + '.csv用时%.8s' % (file_end_time - file_start_time))
#     print('=========================')

# 没有限制,最终采用的代码

In [14]:
def data_clean_new(name):
    _ = open("D:\\IDEA\\情感分析论文\\清洗后的数据\\" + name + '_allcleaned' + '.csv',
             'w',
             encoding='utf-8')
    _.close()
    print(name + '_clean.csv已创建')

    # 创建写入器
    csvfile = open("D:\\IDEA\\情感分析论文\\清洗后的数据\\" + name + '_allcleaned' + '.csv',
                   'a+',
                   newline='',
                   encoding='utf-8')
    # a+表示以追加模式写入，如果用w会覆盖掉原来的数据
    csv_write = csv.writer(csvfile)

    # 读取并写入
    print(name + '_allclean.csv正在写入')
    f = open('D:\\IDEA\\情感分析论文\\' + 'half_data' + '\\' + name + '_city.csv',
             encoding='utf-8')
    csv_read = csv.reader(f)
    for i, line in enumerate(csv_read):
        city, province = get_position(line[-1].split(','), i)
        NEW_LINE = line + [city] + [province]
        _ = csv_write.writerow(NEW_LINE)
        if i % 10000 == 0:
            print('第%d条' % i)
    csvfile.close()
    f.close()
    print(name + '_allclean.csv写入完成')

In [191]:
file_name = ['2019-12','2020-01','2020-02']
for name in file_name:
    file_start_time = time.time()
    data_clean_new(name)
    file_end_time = time.time()
    print('清洗' + name + '.csv用时%.8s' % (file_end_time - file_start_time))
    print('=========================')

2019-12_clean.csv已创建
2019-12_allclean.csv正在写入


NameError: name 'get_position' is not defined

# 读取数据，合并了经纬坐标

In [82]:
os.chdir('D:/IDEA/情感分析论文/清洗后的数据')
file_lst=np.array(os.listdir('D:/IDEA/情感分析论文/清洗后的数据'))[1:]
file_lst[[0,2,4]]
lst=[]
for file in file_lst[[0,2,4]]:
    df=pd.read_csv(file,header=None)
    lst.append(df)
    
#'2020-02-23'
df1=pd.concat(lst,axis=0)
df1.head()

array(['2019-12_allcleaned.csv', '2020-01_allcleaned.csv',
       '2020-02_allcleaned.csv'], dtype='<U22')

,0,1,2,3,4,5,6,7,8,9,10
0,IiF8jBKXO,1587739021,2019-12-01 00:08:46,0,0,0,艾滋病是一种危害性极大的传染病，由感染艾滋病病毒（HIV病毒）引起。HIV是一种能攻击人体免...,NaN,"125.26175,43.82016",长春市,吉林省
1,IiF8px6dn,1587936682,2019-12-01 00:09:00,24,1,7,HAPPY HOMECOMING 内江 显示地图,NaN,"104.67524,29.538946",内江市,四川省
2,IiFO772UY,1588006270,2019-12-01 01:51:44,34,0,9,刚不小心被狗咬了一口🐶 请问需要打狂犬疫苗么～ 临沂·临沭县 显示地图,NaN,"118.653564,34.91798",临沂市,山东省
3,IiGZdCQgX,1587866750,2019-12-01 04:51:51,3,0,2,不失眠但多梦 算不上早睡但绝对是早起 牡丹江·绥芬河市 显示地图,NaN,"131.152542,44.412312",牡丹江市,黑龙江省
4,IiHq82dV1,1587983445,2019-12-01 05:58:07,7,0,10,回国前例行shopping 海关蜀黍求放过👮🏻♂️👮🏼♂️👮🏽♂️👮🏾♂️👮🏿♂️ 美国...,NaN,"-71.135239,42.350685",Boston,Massachusetts


In [83]:
df1=df1[~df1[9].isnull()]
df1.reset_index(drop=True,inplace=True)
df1.shape
df1.head()

(428259, 11)

,0,1,2,3,4,5,6,7,8,9,10
0,IiF8jBKXO,1587739021,2019-12-01 00:08:46,0,0,0,艾滋病是一种危害性极大的传染病，由感染艾滋病病毒（HIV病毒）引起。HIV是一种能攻击人体免...,NaN,"125.26175,43.82016",长春市,吉林省
1,IiF8px6dn,1587936682,2019-12-01 00:09:00,24,1,7,HAPPY HOMECOMING 内江 显示地图,NaN,"104.67524,29.538946",内江市,四川省
2,IiFO772UY,1588006270,2019-12-01 01:51:44,34,0,9,刚不小心被狗咬了一口🐶 请问需要打狂犬疫苗么～ 临沂·临沭县 显示地图,NaN,"118.653564,34.91798",临沂市,山东省
3,IiGZdCQgX,1587866750,2019-12-01 04:51:51,3,0,2,不失眠但多梦 算不上早睡但绝对是早起 牡丹江·绥芬河市 显示地图,NaN,"131.152542,44.412312",牡丹江市,黑龙江省
4,IiHq82dV1,1587983445,2019-12-01 05:58:07,7,0,10,回国前例行shopping 海关蜀黍求放过👮🏻♂️👮🏼♂️👮🏽♂️👮🏾♂️👮🏿♂️ 美国...,NaN,"-71.135239,42.350685",Boston,Massachusetts


# 判断是否为关于疫情的博文

## 1. 首先三条原则做删除
'(记者ssfsdf)'


视频，显示，地图都有分，位置也有
类型：如果有的微博视频删除三个,其他删除两个(原因在于这些地址信息会干扰内容判断)

多个网址的，剩余半个网址的
1.1 的视频，网址，地图


1.2 位置，的视频，显示地图*


1.3 网址，显示地图


1.4 位置，显示地图*


In [7]:
# 句子切分
def cut_sentence(words):
    words = words.encode('utf-8').decode('utf-8')
    start = 0
    i = 0
    token = 'meaningless'
    sents = []
    punt_list = ',.!?;~，。！？；～… '.encode('utf-8').decode('utf-8')
    #print "punc_list", punt_list
    for word in words:
        #print "word", word
        if word not in punt_list:  # 如果不是标点符号
            #print "word1", word
            i += 1
            token = list(words[start:i + 2]).pop()
            #print "token:", token
        elif word in punt_list and token in punt_list:  # 处理省略号
            #print "word2", word
            i += 1
            token = list(words[start:i + 2]).pop()
            #print "token:", token
        else:
            #print "word3", word
            sents.append(words[start:i + 1])  # 断句
            start = i + 1
            i += 1
    if start < len(words):  # 处理最后的部分
        sents.append(words[start:])
    return sents

# 切分后重组
combine=lambda _lst:''.join(_lst)
shipin=lambda string: len(re.findall(r'(的微博视频)|(的秒拍视频)',string))

In [77]:
# 提取超话内容
def get_super(string):
    # ？表示贪婪匹配
    return combine(re.findall('#.*?#',string))

def cut_for_clean(string):
    # 移除超话内容
    string=re.sub('#.*?#','',string)
    _s=pd.Series(cut_sentence(string))
    # 删除@语段
    _s=_s[~_s.str.startswith('@')].tolist()
    if shipin(string)!=0:
        return combine(_s[:-3])
    else:
        return combine(_s[:-2])

In [85]:
df_user_content=df1.copy()
df_user_content[6]=df_user_content[6].apply(cut_for_clean)
columns=['_id','crawl_time','created_at','like_num','repost_num','comment_num','content','origin_weibo','location_map_info', 'city', 'province']
df_user_content.columns=columns
df_user_content.head()

,0,1,2,3,4,5,6,7,8,9,10
0,IiF8jBKXO,1587739021,2019-12-01 00:08:46,0,0,0,艾滋病是一种危害性极大的传染病，由感染艾滋病病毒（HIV病毒）引起。HIV是一种能攻击人体免...,NaN,"125.26175,43.82016",长春市,吉林省
1,IiF8px6dn,1587936682,2019-12-01 00:09:00,24,1,7,HAPPY HOMECOMING,NaN,"104.67524,29.538946",内江市,四川省
2,IiFO772UY,1588006270,2019-12-01 01:51:44,34,0,9,刚不小心被狗咬了一口🐶 请问需要打狂犬疫苗么～,NaN,"118.653564,34.91798",临沂市,山东省
3,IiGZdCQgX,1587866750,2019-12-01 04:51:51,3,0,2,不失眠但多梦 算不上早睡但绝对是早起,NaN,"131.152542,44.412312",牡丹江市,黑龙江省
4,IiHq82dV1,1587983445,2019-12-01 05:58:07,7,0,10,回国前例行shopping 海关蜀黍求放过👮🏻♂️👮🏼♂️👮🏽♂️👮🏾♂️👮🏿♂️,NaN,"-71.135239,42.350685",Boston,Massachusetts


In [256]:
# timeStamp = df[1].values[123]
# timeArray = time.localtime(timeStamp)
# otherStyleTime = time.strftime("%Y-%m-%d", timeArray)
# otherStyleTime

## 2. 其次根据长度信息剔除商业机构和新闻媒体以及海外用户

In [168]:
def condition_1(df, NUM=50):
    # df = df[df[['like_num', 'repost_num', 'comment_num']].sum(axis=1) > 2]
    # 从用户主题的角度判断是否为关于疫情的内容
    
    # 商业机构，新闻媒体，普通用户
    # 我们认为新闻媒体总是客观的，不包含情绪，即使内容关于疫情，但未提供情绪信息，且关注点在于用户
    # 商业机构总是包含链接，内容包含链接，且文字量较大
    condition_num = (df['comment_num'] <=
                     NUM) & (df['like_num'] <= NUM) & (df['repost_num'] <= NUM)

    # 非转发微博，字数限制在140以内
    condition1 = ((df['content'].apply(len) <= 140).values
                  & df['origin_weibo'].isnull().values)
    
    # 转发微博，字数限制在280以内
    condition2 = ((df['content'].apply(len) <= 280).values
                  & df['origin_weibo'].notnull().values)
    # post条件
    condition_post = (condition1) | (condition2)
    
    # 最终条件
    condition=condition_num&condition_post
    return condition

In [257]:
p = re.compile(r'^[a-zA-Z]')
def foreign_prov(_prov):
    if p.match(_prov)==None:
        return False
    else:
        return True
# 剔除国外城市
df_user_content['province']=df_user_content['province'].astype(str)
df_temp=df_user_content[~(df_user_content['province'].apply(foreign_prov))]

# 剔除商业机构和新闻
df_content=df_temp[condition_1(df_temp)]
df_content.reset_index(drop=True,inplace=True)
df_content.shape
df_content.head()

(354686, 11)

,_id,crawl_time,created_at,like_num,repost_num,comment_num,content,origin_weibo,location_map_info,city,province
0,IiF8px6dn,1587936682,2019-12-01 00:09:00,24,1,7,HAPPY HOMECOMING,NaN,"104.67524,29.538946",内江市,四川省
1,IiFO772UY,1588006270,2019-12-01 01:51:44,34,0,9,刚不小心被狗咬了一口🐶 请问需要打狂犬疫苗么～,NaN,"118.653564,34.91798",临沂市,山东省
2,IiGZdCQgX,1587866750,2019-12-01 04:51:51,3,0,2,不失眠但多梦 算不上早睡但绝对是早起,NaN,"131.152542,44.412312",牡丹江市,黑龙江省
3,IiIso8VpQ,1587987156,2019-12-01 08:36:27,0,0,0,12月你好,NaN,"131.1525458345,44.412310280301",牡丹江市,黑龙江省
4,IiIuK8PFt,1588026123,2019-12-01 08:42:15,2,0,5,外面看着好冷清,NaN,"119.303543,26.078131",福州市,福建省


In [387]:
df_content.to_csv("论文数据/df_content.csv",index=False)

#####################################################################################################

#####################################################################################################

#####################################################################################################

#####################################################################################################

#####################################################################################################

#####################################################################################################

In [258]:
## 提取超话内容,只有df1中有超话，并且为了和df_content匹配，需要应用condition_1
super_content=df1['content'][condition_1(df_user_content)].apply(get_super)
super_content.reset_index(drop=True,inplace=True)
(super_content.apply(len)==0).sum()
super_content.head()

207581

0    
1    
2    
3    
4    
Name: content, dtype: object